In [1]:
from langchain_community.llms import Ollama

# Specify the remote server's URL
llm = Ollama(model="phi4:latest", base_url="http://127.0.0.1:11434")

/tmp/ipykernel_161669/4093896791.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="phi4:latest", base_url="http://127.0.0.1:11434")


In [2]:
llm.invoke('hi')

'Hello! How can I assist you today? If you have any questions or need information, feel free to ask.'

In [3]:
llm.invoke('سلام. خوبی؟')

'سلام! بله، خیلی خوبم، صبر کنید چطوری می\u200cشود؟ شما چطور هستید؟'

In [4]:
from langchain.chains import LLMMathChain

llm_math = LLMMathChain.from_llm(llm, verbose=True)

In [5]:
llm_math.prompt.template

'Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'

In [8]:
# llm_math.run("what is 13 raised to the .3432 power?")

In [6]:
i = 0
example_query = "What is 551368 divided by 82"
llm_math.invoke(example_query)



> Entering new LLMMathChain chain...
What is 551368 divided by 82To solve the problem of dividing 551368 by 82 using Python's `numexpr` library, we first need to translate the mathematical expression into a format that can be executed by `numexpr.evaluate`. Here’s how you can do it:

```text
551368 / 82
```

Now, let's evaluate this expression using `numexpr`:

...numexpr.evaluate("551368 / 82")...

The output of running this code will give us the result of the division.

Answer: 6722.0

So, 551368 divided by 82 is 6722.0.
> Finished chain.


{'question': 'What is 551368 divided by 82',
 'answer': 'Answer:  6722.0\n\nSo, 551368 divided by 82 is 6722.0.'}

## understand which formula is selected

In [2]:
from langchain.chains import LLMChain
from langchain.chains.router import LLMRouterChain
from langchain.prompts import PromptTemplate

In [ ]:
import json
import re
from langchain.schema import BaseOutputParser


In [3]:
# Define prompt templates for each formula chain
house_price_prompt = PromptTemplate(
    input_variables=["question"],
    template="This question asks about a house price. Apply Formula1: {question}"
)
other_task_prompt = PromptTemplate(
    input_variables=["question"],
    template="This question seems to be about another task. Apply Formula2: {question}"
)

In [4]:
# Create LLM chains for each formula
house_price_chain = LLMChain(llm=llm, prompt=house_price_prompt)
other_task_chain = LLMChain(llm=llm, prompt=other_task_prompt)

/tmp/ipykernel_161669/207616203.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  house_price_chain = LLMChain(llm=llm, prompt=house_price_prompt)


In [15]:
# from langchain.chains.router import LLMRouterChain
# from langchain.prompts import PromptTemplate

# # Define a router prompt template that instructs the LLM on how to choose a chain.
# router_prompt = PromptTemplate(
#     input_variables=["question"],
#     template=(
#         "You are a routing assistant. Given the question below, "
#         "if it is about house pricing, reply with 'house_price'. Otherwise, reply with 'other_task'.\n"
#         "Question: {question}"
#     )
# )

# # Create the router chain using the supported factory method
# router_chain = LLMRouterChain.from_llm(
#     llm=llm,  # your LLM instance for routing
#     prompt=router_prompt,
#     destination_chains={
#         "house_price": house_price_chain,
#         "other_task": other_task_chain,
#     }
# )

In [16]:
# import json
# from langchain.schema import BaseOutputParser

# class CustomRouterOutputParser(BaseOutputParser):
#     def parse(self, text: str) -> dict:
#         try:
#             result = json.loads(text)
#             # Ensure the required keys are present
#             if "destination" not in result or "next_inputs" not in result:
#                 raise ValueError("Output JSON is missing required keys.")
#             return result
#         except json.JSONDecodeError as e:
#             raise ValueError(f"Could not parse output as JSON: {text}") from e

#     @property
#     def _type(self) -> str:
#         return "custom_router_output_parser"


In [17]:
# from langchain.chains.router.llm_router import LLMRouterChain
# from langchain.prompts import PromptTemplate

# # Instantiate your custom output parser
# router_output_parser = CustomRouterOutputParser()

# # Define the router prompt with the custom output parser attached
# router_prompt = PromptTemplate(
#     input_variables=["question"],
#     template=(
#         "You are a routing assistant. Given the question below, "
#         "reply with a JSON string that includes the keys 'destination' and 'next_inputs'. "
#         "If the question is about house pricing, set 'destination' to 'house_price'. "
#         "Otherwise, set it to 'other_task'.\n"
#         "Question: {question}"
#     ),
#     output_parser=router_output_parser
# )

# # Create the router chain using your custom prompt
# router_chain = LLMRouterChain.from_llm(
#     llm=llm,  # your LLM instance for routing
#     prompt=router_prompt,
#     destination_chains={
#         "house_price": house_price_chain,
#         "other_task": other_task_chain,
#     }
# )


In [25]:
# from langchain.chains.router.output_parser import RouterOutputParser

In [14]:
# from langchain.chains.router.llm_router import LLMRouterChain
# from langchain.prompts import PromptTemplate
# from langchain.output_parsers import RouterOutputParser

# # Create an output parser for the router chain
# router_output_parser = RouterOutputParser()

# # Define the router prompt and attach the output parser
# router_prompt = PromptTemplate(
#     input_variables=["question"],
#     template=(
#         "You are a routing assistant. Given the question below, "
#         "reply with a JSON string that includes the keys 'destination' and 'next_inputs'. "
#         "If the question is about house pricing, set 'destination' to 'house_price'. "
#         "Otherwise, set it to 'other_task'.\n"
#         "Question: {question}"
#     ),
#     output_parser=router_output_parser
# )

# # Create the router chain using the updated prompt
# router_chain = LLMRouterChain.from_llm(
#     llm=llm,  # your LLM instance for routing
#     prompt=router_prompt,
#     destination_chains={
#         "house_price": house_price_chain,
#         "other_task": other_task_chain,
#     }
# )

In [13]:
# # Create a router chain that routes based on the question content
# router_chain = LLMRouterChain.from_llm_and_prompts(
#     llm=llm,  
#     destination_chains={
#         "house_price": house_price_chain,
#         "other_task": other_task_chain,
#     },
#     # default_chain=default_chain  # if applicable 
# )


In [ ]:

def clean_keys(d: dict) -> dict:
    new_d = {}
    for key, value in d.items():
        # Strip whitespace and then remove surrounding quotes if present
        new_key = key.strip()
        if new_key.startswith('"') and new_key.endswith('"'):
            new_key = new_key[1:-1]
        if isinstance(value, dict):
            new_d[new_key] = clean_keys(value)
        else:
            new_d[new_key] = value
    return new_d

class CustomRouterOutputParser(BaseOutputParser):
    def parse(self, text: str) -> dict:
        # Extract the first JSON object from the text using regex
        match = re.search(r"(\{.*\})", text, re.DOTALL)
        if not match:
            raise ValueError("No JSON object found in text.")
        json_str = match.group(1)
        try:
            result = json.loads(json_str)
        except json.JSONDecodeError as e:
            raise ValueError(f"Error decoding JSON: {json_str}") from e
        # Clean the keys of the parsed JSON
        result = clean_keys(result)
        # Validate required keys
        if "destination" not in result:
            raise ValueError("Output JSON is missing the 'destination' key.")
        if "next_inputs" not in result:
            raise ValueError("Output JSON is missing the 'next_inputs' key.")
        if not isinstance(result["next_inputs"], dict):
            raise ValueError("The 'next_inputs' key must be a dictionary.")
        if "question" not in result["next_inputs"]:
            raise ValueError("The 'next_inputs' dictionary must include the 'question' key.")
        return result

    @property
    def _type(self) -> str:
        return "custom_router_output_parser"


In [19]:
from langchain.chains.router.llm_router import LLMRouterChain
from langchain.prompts import PromptTemplate

router_output_parser = CustomRouterOutputParser()

router_prompt = PromptTemplate(
    input_variables=["question"],
    template=(
        "You are a routing assistant. Given the following question, decide which destination chain to use and return a valid JSON object with exactly two keys:\n"
        "1. destination: either \"house_price\" (if the question is about house pricing) or \"other_task\" for all other cases.\n"
        "2. next_inputs: a dictionary that includes the key \"question\" mapping to the original question text.\n"
        "For example, if the question is about house pricing, return:\n"
        "{{\"destination\": \"house_price\", \"next_inputs\": {{\"question\": \"What is the price of the house?\"}}}}\n"
        "Now, answer for the question: {question}"
    ),
    output_parser=router_output_parser  # your custom parser
)

router_chain = LLMRouterChain.from_llm(
    llm=llm,  # your routing LLM instance
    prompt=router_prompt,
    destination_chains={
        "house_price": house_price_chain,
        "other_task": other_task_chain,
    }
)


In [20]:
input_text = "please tell me about the price of the your house?"
result = router_chain({"question": input_text})
print(result)


{'question': 'please tell me about the price of the your house?', 'destination': 'house_price', 'next_inputs': {'question': 'please tell me about the price of your house?'}}


## New solution

In [24]:
import json
import re
from langchain.schema import BaseOutputParser
from langchain.chains.router.llm_router import LLMRouterChain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
# from langchain.llms import OpenAI  # Replace with your actual LLM (ollama, phi4, etc.)


In [28]:
# -------------------------------
# Custom Output Parser Definition
# -------------------------------
def clean_keys(d: dict) -> dict:
    """Recursively clean dictionary keys by stripping whitespace and extra quotes."""
    new_d = {}
    for key, value in d.items():
        new_key = key.strip()
        if new_key.startswith('"') and new_key.endswith('"'):
            new_key = new_key[1:-1]
        if isinstance(value, dict):
            new_d[new_key] = clean_keys(value)
        else:
            new_d[new_key] = value
    return new_d

class CustomRouterOutputParser(BaseOutputParser):
    def parse(self, text: str) -> dict:
        # Extract the first JSON object from the text
        match = re.search(r"(\{.*\})", text, re.DOTALL)
        if not match:
            raise ValueError("No JSON object found in text.")
        json_str = match.group(1)
        try:
            result = json.loads(json_str)
        except json.JSONDecodeError as e:
            raise ValueError(f"Error decoding JSON: {json_str}") from e
        # Clean the keys to remove any extra quotes
        result = clean_keys(result)
        # Validate required keys
        if "destination" not in result:
            raise ValueError("Output JSON is missing the 'destination' key.")
        if "next_inputs" not in result:
            raise ValueError("Output JSON is missing the 'next_inputs' key.")
        if not isinstance(result["next_inputs"], dict):
            raise ValueError("The 'next_inputs' key must be a dictionary.")
        if "question" not in result["next_inputs"]:
            raise ValueError("The 'next_inputs' dictionary must include the 'question' key.")
        return result

    @property
    def _type(self) -> str:
        return "custom_router_output_parser"

# Instantiate the custom output parser
router_output_parser = CustomRouterOutputParser()

In [29]:
# -------------------------------------------
# Update: Define the Router Prompt (with escaped braces)
# -------------------------------------------
# CHANGED: The prompt now includes the "loan_calculation" destination.
router_prompt = PromptTemplate(
    input_variables=["question"],
    template=(
        "You are a routing assistant. Given the following question, decide which formula chain to use and return a valid JSON object with exactly two keys:\n"
        "1. destination: one of 'formula_1', 'formula_2', or 'loan_calculation'.\n"
        "   - Use 'formula_1' for questions about house pricing.\n"
        "   - Use 'loan_calculation' for questions about calculating remaining money for loans.\n"
        "   - Use 'formula_2' for all other tasks.\n"
        "2. next_inputs: a dictionary that includes the key 'question' mapping to the original question text.\n"
        "For example, if the question is 'What is the price of the house?', return:\n"
        "{{\"destination\": \"formula_1\", \"next_inputs\": {{\"question\": \"What is the price of the house?\"}}}}\n"
        "Now, answer for the question: {question}"
    ),
    output_parser=router_output_parser
)

In [30]:
# --------------------------------------------------------
# Create Dummy Formula Chains for Each Task/Formula
# --------------------------------------------------------
# llm = OpenAI(temperature=0)  # Replace with your preferred LLM instance

# Formula 1: House Pricing (unchanged)
formula_1_prompt = PromptTemplate(
    input_variables=["question"],
    template="Formula 1 (House Pricing): Given the question: {question}, compute the house price using the relevant pricing formula."
)
formula_1_chain = LLMChain(llm=llm, prompt=formula_1_prompt, verbose=True)

# Formula 2: Generic Task (unchanged)
formula_2_prompt = PromptTemplate(
    input_variables=["question"],
    template="Formula 2: Given the question: {question}, compute the answer using a generic approach."
)
formula_2_chain = LLMChain(llm=llm, prompt=formula_2_prompt, verbose=True)

# NEW: Loan Calculation Chain
# This chain is used when the question is about calculating how much money to pay for loans.
loan_calculation_prompt = PromptTemplate(
    input_variables=["question"],
    template=(
        "Loan Calculation Expert: For the question: {question}\n"
        "Use the formula: total_amount - (number_of_paid_loans * amount_per_loan)\n"
        "to calculate the amount of money required to pay all the loans. \n"
        "Here, total_amount is the total money received by the user, "
        "number_of_paid_loans is the number of loans paid, and "
        "amount_per_loan is the cost of each paid loan. \n"
        "Explain the formula and show an example calculation."
    )
)
loan_calculation_chain = LLMChain(llm=llm, prompt=loan_calculation_prompt, verbose=True)

# ---------------------------------------------------------------
# Update: Create the Router Chain with the Three Destination Chains
# ---------------------------------------------------------------
# CHANGED: The destination_chains dictionary now includes the key "loan_calculation"
router_chain = LLMRouterChain.from_llm(
    llm=llm,
    prompt=router_prompt,
    destination_chains={
        "formula_1": formula_1_chain,
        "formula_2": formula_2_chain,
        "loan_calculation": loan_calculation_chain,  # NEW destination
    }
)


In [34]:
# Create the mapping for destination chains (separate from the router chain)
destination_chains = {
    "formula_1": formula_1_chain,
    "formula_2": formula_2_chain,
    "loan_calculation": loan_calculation_chain,
}

# -------------------------------------------
# Example Usage: A Loan Calculation Question
# -------------------------------------------
input_text = "How much money should I pay to pay all my loans to the bank?"
# input_text = "چه مقدار پول باید بدم به بانک تا وام من تسویه بشه؟"
# First, use the router chain to determine which formula chain to use.
router_result = router_chain({"question": input_text})
print("Router output:", router_result)

# Extract destination and next_inputs from the router output.
destination = router_result["destination"]
next_inputs = router_result["next_inputs"]

# Use our separate mapping to call the selected destination chain.
if destination in destination_chains:
    final_result = destination_chains[destination](next_inputs)
    print(f"Final result from '{destination}':", final_result)
else:
    print("Destination not found:", destination)


Router output: {'question': 'How much money should I pay to pay all my loans to the bank?', 'destination': 'loan_calculation', 'next_inputs': {'question': 'How much money should I pay to pay all my loans to the bank?'}}


> Entering new LLMChain chain...
Prompt after formatting:
Loan Calculation Expert: For the question: How much money should I pay to pay all my loans to the bank?
Use the formula: total_amount - (number_of_paid_loans * amount_per_loan)
to calculate the amount of money required to pay all the loans. 
Here, total_amount is the total money received by the user, number_of_paid_loans is the number of loans paid, and amount_per_loan is the cost of each paid loan. 
Explain the formula and show an example calculation.

> Finished chain.
Final result from 'loan_calculation': {'question': 'How much money should I pay to pay all my loans to the bank?', 'text': "### Explanation of the Formula\n\nThe formula used to calculate how much money you should pay to settle all your loans i